In [1]:
from itertools import product
import operator        
import datetime
import warnings
warnings.filterwarnings('ignore')

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd

import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, QuantileTransformer
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, lars_path, ElasticNet, ElasticNetCV, Ridge, RidgeCV
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# pointer to scaler I want to use
use_transform = RobustScaler

def mytransform():
    return QuantileTransformer(output_distribution='normal')

import scipy.stats as stats
from scipy.stats import skew  
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

#!conda install -y -conda-forge  xgboost 
from xgboost import XGBRegressor
from xgboost import plot_importance

#!conda install -y -c conda-forge lightgbm
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#!conda install -y -c districtdatalabs yellowbrick
from yellowbrick.features import JointPlotVisualizer, ParallelCoordinates
from yellowbrick.regressor import AlphaSelection, PredictionError, ResidualsPlot

#!conda install -y -c conda-forge mlxtend
from mlxtend.regressor import StackingCVRegressor

import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

import ipywidgets as widgets
from IPython.display import clear_output
pd.options.display.max_columns=None
#pd.options.display.max_rows=None

numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

print(datetime.datetime.now())



2019-06-09 14:02:06.243513


In [2]:
blend1 = pd.read_csv('data/agehsbarg_mice.csv')
blend1.head()


,Id,SalePrice
0,1461,121118.161176
1,1462,166296.677557
2,1463,185264.902418
3,1464,195140.874098
4,1465,190226.389555


In [3]:
blend2 = pd.read_csv('data/couyang_hybrid.csv')
blend2.head()


,Id,SalePrice
0,1461,120635.089551
1,1462,166995.910189
2,1463,188855.105080
3,1464,201677.418621
4,1465,189999.640754


In [4]:
blend3 = pd.read_csv('data/zavodrobotov_lasso_sol22.csv')
blend3.head()


,Id,SalePrice
0,1461,123869.209797
1,1462,160819.061703
2,1463,178337.500093
3,1464,199800.787062
4,1465,198795.333026


In [5]:
my_sub = pd.read_csv('stackblend3_adjust.csv')
my_sub.head()

,Id,SalePrice
0,1461,120433.359372
1,1462,160638.213375
2,1463,186749.664930
3,1464,197112.879997
4,1465,184015.168940


In [6]:
blend1.rename(columns={'SalePrice': 'blend1'}, inplace=True)
blend2.rename(columns={'SalePrice': 'blend2'}, inplace=True)
blend3.rename(columns={'SalePrice': 'blend3'}, inplace=True)
my_sub.rename(columns={'SalePrice': 'model'}, inplace=True)

In [7]:
mymerge = my_sub.merge(blend1, on='Id') \
    .merge(blend2, on='Id') \
    .merge(blend3, on='Id')
mymerge.head()

,Id,model,blend1,blend2,blend3
0,1461,120433.359372,121118.161176,120635.089551,123869.209797
1,1462,160638.213375,166296.677557,166995.910189,160819.061703
2,1463,186749.664930,185264.902418,188855.105080,178337.500093
3,1464,197112.879997,195140.874098,201677.418621,199800.787062
4,1465,184015.168940,190226.389555,189999.640754,198795.333026


In [8]:
for col in ['model', 'blend1', 'blend2', 'blend3']:
    mymerge[col] = np.log1p(mymerge[col])

In [9]:
mymerge['SalePrice']=mymerge[['model','blend2','blend3']].mean(axis=1)

for col in ['model', 'blend1', 'blend2', 'blend3', 'SalePrice']:
    mymerge[col] = np.expm1(mymerge[col])
    
mymerge.head()

,Id,model,blend1,blend2,blend3,SalePrice
0,1461,120433.359372,121118.161176,120635.089551,123869.209797,121635.760055
1,1462,160638.213375,166296.677557,166995.910189,160819.061703,162791.131532
2,1463,186749.664930,185264.902418,188855.105080,178337.500093,184590.953149
3,1464,197112.879997,195140.874098,201677.418621,199800.787062,199521.556481
4,1465,184015.168940,190226.389555,189999.640754,198795.333026,190840.639929


In [10]:
mymerge[['Id', 'SalePrice']].to_csv('superblend.csv', index=False)
